In [112]:
import json
import pandas as pd
#model_name = ["bge-large-en-v1.5" , "contriever", "gte-base", "multilingual-e5-large"]
#dataset_name = ["nfcorpus", "trec-covid"]

In [140]:
def cal_ndcg(
    model_name,
    dataset_name
):
    pass

def cal_recall(
    model_name, 
    retrieval_results, 
    gt
):
    recall = 0
    count = 0
    for qid, ret_ids in retrieval_results.items():
        if qid not in gt:
            continue
        gt_ids = gt[qid]
        inter = len(set(ret_ids) & set(gt_ids))
        recall += inter / (len(gt_ids))
        count += 1
    print(f"Model: {model_name} / Metric: {round(recall / count * 100, 2)}")



In [146]:
def print_results(
    dataset_name,
    top_k=100
):
    qrels = pd.read_csv(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/qrels/{mode}.tsv", sep="\t")
    qrels = qrels[qrels["score"] >= 1].sort_values("corpus-id")

    # Get ground truth results
    gt = {}
    for k in qrels["query-id"].unique():
        qrels_ = qrels[qrels["query-id"] == k]
        v = qrels_["corpus-id"].tolist()
        gt[str(k)] = v

    # For each model, calculate the metric
    for model_name in ["bge-large-en-v1.5" , "contriever", "gte-base", "multilingual-e5-large"]:
        with open(f"{model_name}/{dataset_name}.jsonl") as f:
            results = [json.loads(i) for i in f.readlines()]
        retrieval_results = {str(i["_id"]): [ret["_id"] for ret in i["retrieval"]] for i in results}
        #cal_ndcg(model_name, retrieval_results, gt)
        cal_recall(model_name, retrieval_results, gt)

In [145]:
print_results("nfcorpus")

Total: 3237 retrievals
Model: bge-large-en-v1.5 / Metric: 35.41
Total: 3237 retrievals
Model: contriever / Metric: 26.22
Total: 3237 retrievals
Model: gte-base / Metric: 34.58
Total: 3237 retrievals
Model: multilingual-e5-large / Metric: 29.58


In [147]:
print_results("trec-covid")

Model: bge-large-en-v1.5 / Metric: 12.61
Model: contriever / Metric: 1.56
Model: gte-base / Metric: 13.99
Model: multilingual-e5-large / Metric: 12.09


## Check trec-covid

In [149]:
# Load datasets
org = pd.read_csv(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/qrels/test.tsv", sep="\t")
v2 = pd.read_csv(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid-v2/qrels/test.tsv", sep="\t")
beir = pd.read_csv(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid-beir/qrels/test.tsv", sep="\t")

In [162]:
len(beir["corpus-id"].unique())

36016

In [151]:
v2

,query-id,corpus-id,score
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1
...,...,...,...
56723,50,zv4nbz9p,2
56724,50,zvop8bxh,2
56725,50,zwf26o63,1
56726,50,zwsvlnwe,0


In [152]:
beir

,query-id,corpus-id,score
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1
...,...,...,...
67311,50,f8aolprb,0
67312,50,we7wv6so,0
67313,50,t35voozy,0
67314,50,fb8a6evc,0


In [163]:
df1 = v2
df2 = beir
ds1 = set(tuple(line) for line in df1.values)
ds2 = set(tuple(line) for line in df2.values)
df = pd.DataFrame(list(ds2.difference(ds1)), columns=df2.columns)

5413

In [ ]:
# For each model, calculate the metric
dataset_name = "trec-covid"
for model_name in ["bge-large-en-v1.5" , "contriever", "gte-base", "multilingual-e5-large"]:
    with open(f"{model_name}/{dataset_name}.jsonl") as f:
        results = [json.loads(i) for i in f.readlines()]
    retrieval_results = {str(i["_id"]): [ret["_id"] for ret in i["retrieval"]] for i in results}
    #cal_ndcg(model_name, retrieval_results, gt)
    cal_recall(model_name, retrieval_results, gt)